# Homework 1: Introduction

In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt                     

from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import root_mean_squared_error

import pickle

In [2]:
df = pd.read_parquet("../data/yellow_tripdata_2023-01.parquet")

## Q1. Downloading the data

In [3]:
df.shape

(3066766, 19)

## Q2. Computing duration

In [4]:
df['duration'] = df['tpep_dropoff_datetime'] - df['tpep_pickup_datetime']
df['duration']  = df['duration'].apply(lambda x: x.total_seconds()/60)
df['duration'].std()

42.59435124195458

## Q3. Dropping outliers

In [5]:
(len(df[(df.duration >= 1) & (df.duration <= 60)])/len(df)) * 100

98.1220282212598

## Q4. One-hot encoding


In [6]:
categorical = ['PULocationID','DOLocationID']
numerical = ['trip_distance']

In [7]:
df[categorical] = df[categorical].astype(str)

In [8]:
train_dict = df[categorical + numerical].to_dict(orient='records')
dv = DictVectorizer()
X_train = dv.fit_transform(train_dict)
dimensionality = X_train.shape[1]
dimensionality

519

## Q5. Training a model

In [11]:
def read_df(path):
    df = pd.read_parquet(path)

    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

    df = df[(df.duration >= 1) & (df.duration <= 60)]

    categorical = ['PULocationID', 'DOLocationID']
    numerical = ['trip_distance']

    df[categorical] = df[categorical].astype(str)
    return df


In [12]:
categorical = ['PULocationID', 'DOLocationID']
numerical = ['trip_distance']
target = 'duration'
dv = DictVectorizer()

df_train = read_df('../data/yellow_tripdata_2023-01.parquet')
train_dict = df_train[categorical + numerical].to_dict(orient='records')
X_train = dv.fit_transform(train_dict)
y_train = df_train[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)
y_pred = lr.predict(X_train)

rms = root_mean_squared_error(y_train, y_pred)
print(f"Root Mean Squared Error: {round(rms,2)}")

Root Mean Squared Error: 7.66


## Q6. Evaluating a model

In [13]:
df_val = read_df('../data/yellow_tripdata_2023-02.parquet')
val_dict = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dict)
y_val = df_val[target].values


y_pred = lr.predict(X_val)
rms = root_mean_squared_error(y_val, y_pred)
print(f"Root Mean Squared Error: {round(rms,2)}")

Root Mean Squared Error: 7.82
